In [3]:
!apt-get install -y libzbar0
!pip install pyzbar opencv-python pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-double3
  libgs9 libgs9-common libidn12 libijs-0.35 libilmbase25 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libopenexr25 libv4l-0
  libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x libfftw3-bin libfftw3-dev inkscape poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-t

In [4]:
import gradio as gr
import cv2
import numpy as np
from pyzbar.pyzbar import decode
from PIL import Image
import requests
import pandas as pd
from sklearn.cluster import KMeans
from difflib import get_close_matches
import random

def process_barcode_image(image):
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    barcodes = decode(img_cv)

    if not barcodes:
        return "❌ No barcode detected", None, None

    barcode = barcodes[0].data.decode("utf-8")

    url = f"https://world.openfoodfacts.org/api/v0/product/{barcode}.json"
    response = requests.get(url)

    if response.status_code != 200:
        return f"❌ Failed to fetch product for barcode `{barcode}`", None, None

    data = response.json()
    if data.get("status") != 1:
        return f"❌ Product not found for barcode `{barcode}`", None, None

    product = data["product"]
    product_name = product.get("product_name", "Unknown Product")
    nutriments = product.get("nutriments", {})

    summary = f"### 📦 Product: {product_name}\n\n"
    sugar = float(nutriments.get("sugars_100g", 0))

    if sugar > 30:
        summary += "🍭 **High sugar!** Not the healthiest choice...\n\n"
    elif sugar > 10:
        summary += "🙂 **Moderate sugar level.**\n\n"
    else:
        summary += "💚 **Low sugar!** Good choice!\n\n"

    summary += "**📊 Nutritional Info per 100g:**\n"
    for key, val in nutriments.items():
        if "_100g" in key:
            summary += f"- {key}: {val}\n"

    # ML recommendations
    categories = ["snack", "meal", "beverage", "chocolate"]
    all_products = []
    for cat in categories:
        r = requests.get("https://world.openfoodfacts.org/cgi/search.pl", params={
            "search_terms": cat,
            "search_simple": 1,
            "action": "process",
            "json": 1,
            "page_size": 100
        })
        if r.status_code == 200:
            all_products += r.json().get("products", [])

    rows = []
    for prod in all_products:
        name = prod.get("product_name", "Unknown")
        nutri = prod.get("nutriments", {})
        try:
            row = {
                "product_name": name,
                "energy": float(nutri.get("energy_100g", 0)),
                "sugars": float(nutri.get("sugars_100g", 0)),
                "fat": float(nutri.get("fat_100g", 0)),
                "fiber": float(nutri.get("fiber_100g", 0)),
                "proteins": float(nutri.get("proteins_100g", 0))
            }
            rows.append(row)
        except:
            continue

    df = pd.DataFrame(rows)
    for col in ["energy", "sugars", "fat", "fiber", "proteins"]:
        df[col] = df[col].replace(0, pd.NA)
        df[col] = df[col].fillna(df[col].mean())
    df.dropna(inplace=True)

    kmeans = KMeans(n_clusters=4, random_state=42)
    df["cluster"] = kmeans.fit_predict(df[["energy", "sugars", "fat", "fiber", "proteins"]])

    matches = get_close_matches(product_name, df["product_name"].tolist(), n=1, cutoff=0.5)
    if matches:
        matched_name = matches[0]
        input_row = df[df["product_name"] == matched_name]
        current_cluster = input_row["cluster"].values[0]
        healthy_cluster = df.groupby("cluster")[["sugars", "energy"]].mean().sum(axis=1).idxmin()
        alternatives = df[df["cluster"] == healthy_cluster].sort_values(by=["energy", "sugars"]).head(5)

        return summary, alternatives[["product_name", "energy", "sugars", "fat", "fiber", "proteins"]], barcode
    else:
        return summary + "\n❌ Couldn't find matching product for ML suggestion.", None, barcode

iface = gr.Interface(
    fn=process_barcode_image,
    inputs=gr.Image(label="Upload Barcode Image", type="pil"),
    outputs=[
        gr.Markdown(label="Product Info"),
        gr.Dataframe(label="Suggested Healthier Alternatives"),
        gr.Textbox(label="Detected Barcode")
    ],
    title="📷 Barcode Nutrition Recommender",
    description="Upload a product image with a barcode to get nutrition info and ML-based healthier alternatives."
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c85ffefdf33843cdd7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
